In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict, StratifiedKFold
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, classification_report

In [2]:
url = "https://raw.githubusercontent.com/datasciencedojo/datasets/master/titanic.csv"
df = pd.read_csv(url)
df = df.drop(['PassengerId', 'Name', 'Ticket', 'Cabin'], axis=1)

In [3]:
X = df.drop('Survived', axis=1)
y = df['Survived']

numerical_cols = ['Age', 'Fare', 'SibSp', 'Parch']
categorical_cols = ['Sex', 'Embarked', 'Pclass']

In [4]:
num_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

cat_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer([
    ('num', num_pipeline, numerical_cols),
    ('cat', cat_pipeline, categorical_cols)
])

In [5]:
model_pipeline = Pipeline([
    ('preprocessing', preprocessor),
    ('classifier', LogisticRegression(max_iter=200))
])

In [6]:
# Use Stratified K-Fold for balanced splits
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Accuracy scores
acc_scores = cross_val_score(model_pipeline, X, y, cv=cv, scoring='accuracy')
prec_scores = cross_val_score(model_pipeline, X, y, cv=cv, scoring='precision')
recall_scores = cross_val_score(model_pipeline, X, y, cv=cv, scoring='recall')

print("✅ Cross-Validation Results:")
print(f"Accuracy Scores: {acc_scores}")
print(f"Mean Accuracy: {np.mean(acc_scores):.2f}")
print(f"Mean Precision: {np.mean(prec_scores):.2f}")
print(f"Mean Recall: {np.mean(recall_scores):.2f}")


✅ Cross-Validation Results:
Accuracy Scores: [0.78212291 0.80337079 0.79775281 0.78089888 0.82022472]
Mean Accuracy: 0.80
Mean Precision: 0.75
Mean Recall: 0.70


In [7]:
y_pred = cross_val_predict(model_pipeline, X, y, cv=cv)
print("\n Classification Report:\n")
print(classification_report(y, y_pred))


 Classification Report:

              precision    recall  f1-score   support

           0       0.82      0.86      0.84       549
           1       0.75      0.70      0.73       342

    accuracy                           0.80       891
   macro avg       0.79      0.78      0.78       891
weighted avg       0.80      0.80      0.80       891

